In [6]:
import requests
from bs4 import BeautifulSoup

# Function to scrape a specific page number
def scrape_recipes_page(page_number):
    base_url = "https://recipes.lewagon.com/"
    params = {
        "page": page_number
    }

    response = requests.get(base_url, params=params)

    # Check if the response status is OK (200)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
    else:
        print(f"Error while retrieving page {page_number}.")
        return None

if __name__ == "__main__":
    import os
    
    # Get the total number of pages from environment variable 'NB_PAGES', default is 123
    total_pages = int(os.getenv('NB_PAGES', 123))
    min_recipes_per_page = 4
    max_recipes_per_page = 13

    full_recipe_links = []  # List to store the links
    
    # Loop through each page number up to the specified total_pages
    for page_number in range(1, total_pages + 1):
        # Scrape the current page
        list_soup = scrape_recipes_page(page_number)

        # Check if the page was scraped successfully
        if list_soup:
            # Find all recipe articles on the page
            recipe_articles = list_soup.find_all("div", class_="recipe my-3")
            # Determine the number of recipes to process on this page
            num_recipes_on_page = min(max_recipes_per_page, max(min_recipes_per_page, len(recipe_articles)))
            
            # Loop through the selected number of recipes on the page
            for i in range(num_recipes_on_page):
                # Extract the recipe link from the data-href attribute
                recipe_link = recipe_articles[i]["data-href"]
                full_recipe_link = recipe_link
                full_recipe_links.append(full_recipe_link)  # Add to the list
        else:
            print(f"Unable to retrieve the list of recipes for page {page_number}.")
    

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def extract_preparation_duration(description_text):
    duration_pattern = r"(\d+)\s*minutes"
    duration_match = re.search(duration_pattern, description_text)
    if duration_match:
        preparation_duration = duration_match.group(1)
        return preparation_duration
    return None


def scrape_recipe_info(recipe_url):
    response = requests.get(recipe_url)

    if response.status_code == 200:
        recipe_soup = BeautifulSoup(response.content, "html.parser")

        name = recipe_soup.find("h2").text.strip()

        img_element = recipe_soup.find("img", class_="recipe-img mb-3 mb-md-0")
        img_url = img_element['src'] if img_element else ''

        ingredient_spans = recipe_soup.find_all("p", class_="mb-0")
        ingredients = [span.text.strip() for span in ingredient_spans]

        description_element = recipe_soup.find("p", class_="text-justify recipe-description")
        description_text = description_element.get_text() if description_element else ""
        duration = extract_preparation_duration(description_text)

        difficulty_element = recipe_soup.find("span", class_="recipe-difficulty")
        difficulty = difficulty_element.text.strip() if difficulty_element else ""

        return {
            "name": name,
            "ingredients": ingredients,
            "duration": duration,
            "difficulty": difficulty,
            "img_url": img_url
        }
    else:
        print(f"Erreur lors de la récupération de la recette {recipe_url}.")
        return None

if __name__ == "__main__":
    full_recipe_links = full_recipe_links

    recipes_data = []

    for full_recipe_link in full_recipe_links:
        recipe_info = scrape_recipe_info(full_recipe_link)
        if recipe_info:
            recipes_data.append(recipe_info)
    
    data = pd.DataFrame(recipes_data)
    data['URL'] = full_recipe_links

data

,name,ingredients,duration,difficulty,img_url,URL
0,15 Minute Healthy Dark Chocolate Truffles,"[15 dried dates, pitted, 3/4 cups sliced or sl...",45,Moderate,https://spoonacular.com/recipeImages/631841-55...,https://recipes.lewagon.com/recipes/1410
1,17 Bean White Chicken Chili,"[1 pound Assorted Dry Beans, Cooked According ...",45,Hard,https://spoonacular.com/recipeImages/631843-55...,https://recipes.lewagon.com/recipes/803
2,2 Minute Chocolate Yum,"[2 tablespoons Cocoa Powder, 1 Egg, slightly b...",45,Very hard,https://spoonacular.com/recipeImages/631849-55...,https://recipes.lewagon.com/recipes/877
3,3 Steps for the Best Sheet Pan Chicken and Veg...,"[fresh and dried herbs, 8-14 pieces asparagus,...",35,Hard,https://spoonacular.com/recipeImages/1045246-5...,https://recipes.lewagon.com/recipes/31
4,4 Ingredient Raw Peanut Butter Chocolate Cups,"[2 to 3 tablespoons coconut flour, as needed, ...",45,Very hard,https://spoonacular.com/recipeImages/792705-55...,https://recipes.lewagon.com/recipes/92
...,...,...,...,...,...,...
1464,Zucchini & Basil Frittata,"[12 leaves basil, chopped, 1/2 cup cheddar che...",45,Moderate,https://spoonacular.com/recipeImages/665691-55...,https://recipes.lewagon.com/recipes/14
1465,Zucchini Burgers with Lemon Herb Yogurt Sauce,"[1/2 cup bread crumbs, 1/2 cup Diced cheddar c...",45,Moderate,https://spoonacular.com/recipeImages/665721-55...,https://recipes.lewagon.com/recipes/524
1466,Zucchini Chicken Omelette,"[3 Eggs, 1 tablespoon Water, 150 grams Zucchin...",45,Moderate,https://spoonacular.com/recipeImages/665734-55...,https://recipes.lewagon.com/recipes/485
1467,Zucchini Quiche Appetizers,"[3 cups zucchini, grated, 1 cup Bisquick, 1/2 ...",45,Hard,https://spoonacular.com/recipeImages/665778-55...,https://recipes.lewagon.com/recipes/720


In [14]:
foods = [
    # Meats and Fish
    "chicken", "beef", "pork", "lamb", "veal", "turkey", "duck", "fish", "salmon", "tuna",
    "shrimp", "mussels", "oysters", "bacon", "ham",
    
    # Vegetables
    "carrot", "potato", "tomato", "onion", "garlic", "spinach", "broccoli", "zucchini", "eggplant",
    "bell pepper", "mushroom", "green beans", "asparagus", "cabbage", "celery", "cucumber"
    
    # Fruits
    "apple", "banana", "orange", "strawberries", "grapes", "pear", "pineapple", "kiwi", "melon", "mango", "avocado",
    "cherry", "raspberry", "blueberry", "lime", "lemon", "coconut"
    
    # Dairy Products
    "milk", "cheese", "butter", "yogurt", "cream", "ice cream",
    
    # Grains
    "rice", "pasta", "bread", "flour", "oats", "barley", "quinoa",
    
    # Legumes
    "red beans", "lentils", "chickpeas", "fava beans",
    
    # Herbs and Spices
    "basil", "parsley", "thyme", "rosemary", "oregano", "cumin", "coriander", "cinnamon", "turmeric", "paprika",
    "pepper", "salt",
    
    # Condiments and Sauces
    "ketchup", "mustard", "mayonnaise", "balsamic vinegar", "soy sauce", "tomato sauce", "barbecue sauce",
    "hot sauce", "olive oil", "vinaigrette", "mint"
    
    # Nuts and Seeds
    "almonds", "cashews", "walnuts", "pistachios", "sunflower seeds", "sesame seeds", "chia seeds",
    
    # Sweeteners
    "sugar", "honey", "maple syrup", "stevia", "syrup","chocolate"
    
    # Beverages
    "water", "coffee","fruit juice", "almond milk", "coconut milk",
    
    # Others
    "eggs", "egg", "almond flour", "coconut flour", "yeast", "sourdough starter"
]


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

def clean_ingredient(ingredient):
    cleaned_ingredient = re.sub(r"\d+|/", "", ingredient).strip()
    return cleaned_ingredient

# Appliquer la fonction à chaque ingrédient dans la colonne "ingredients"
data['ingredients'] = data['ingredients'].apply(lambda ingredients: [clean_ingredient(ingredient) for ingredient in ingredients])

# Téléchargement des stopwords en anglais si ce n'est pas déjà fait
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Prétraitement des ingrédients (mise en minuscules, suppression de la ponctuation et des stopwords)
def preprocess_ingredients(ingredient_list):
    processed_ingredients = []
    removed_stopwords = []
    for ingredient in ingredient_list:
        ingredient_lower = ingredient.lower().replace(',', '').replace('.', '')
        if ingredient_lower not in stop_words:
            processed_ingredients.append(ingredient_lower)
        else:
            removed_stopwords.append(ingredient_lower)
    return processed_ingredients, removed_stopwords

data['ingredients_processed'], data['removed_stopwords'] = zip(*data['ingredients'].apply(preprocess_ingredients))

def real_ingredients(ingredients):
    cleaned_ingredients = []
    for ingredient in ingredients:
        # Supprimer les parenthèses collées à un mot
        ingredient = re.sub(r'(\w+)\)', r'\1', ingredient)
        
        words = ingredient.split()
        new_words = [word for word in words if any(food in word for food in foods)]
        cleaned_ingredient = ', '.join(new_words)
        if cleaned_ingredient:  # Vérifier si la chaîne n'est pas vide
            cleaned_ingredients.append(cleaned_ingredient)
    return cleaned_ingredients

# Appliquer la fonction à la colonne d'ingrédients dans le dataframe
data['cleaned_ingredients'] = data['ingredients_processed'].apply(real_ingredients)

data

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cassienbabey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,name,ingredients,duration,difficulty,img_url,URL,ingredients_processed,removed_stopwords,cleaned_ingredients
0,15 Minute Healthy Dark Chocolate Truffles,"[dried dates, pitted, cups sliced or slivered ...",45,Moderate,https://spoonacular.com/recipeImages/631841-55...,https://recipes.lewagon.com/recipes/1410,"[dried dates pitted, cups sliced or slivered a...",[],"[honey, salt]"
1,17 Bean White Chicken Chili,"[pound Assorted Dry Beans, Cooked According to...",45,Hard,https://spoonacular.com/recipeImages/631843-55...,https://recipes.lewagon.com/recipes/803,[pound assorted dry beans cooked according to ...,[],"[chicken, chicken, pepper, onion, garlic, toma..."
2,2 Minute Chocolate Yum,"[tablespoons Cocoa Powder, Egg, slightly beate...",45,Very hard,https://spoonacular.com/recipeImages/631849-55...,https://recipes.lewagon.com/recipes/877,"[tablespoons cocoa powder, egg slightly beaten...",[],"[egg, flour, salt, sugar]"
3,3 Steps for the Best Sheet Pan Chicken and Veg...,"[fresh and dried herbs, - pieces asparagus, ba...",35,Hard,https://spoonacular.com/recipeImages/1045246-5...,https://recipes.lewagon.com/recipes/31,"[fresh and dried herbs, - pieces asparagus, ba...",[],"[asparagus, zucchini, chicken, cheese, salt, p..."
4,4 Ingredient Raw Peanut Butter Chocolate Cups,"[to tablespoons coconut flour, as needed, oz ...",45,Very hard,https://spoonacular.com/recipeImages/792705-55...,https://recipes.lewagon.com/recipes/92,"[to tablespoons coconut flour as needed, oz (...",[],"[flour, syrup, butter]"
...,...,...,...,...,...,...,...,...,...
1464,Zucchini & Basil Frittata,"[leaves basil, chopped, cup cheddar cheese, eg...",45,Moderate,https://spoonacular.com/recipeImages/665691-55...,https://recipes.lewagon.com/recipes/14,"[leaves basil chopped, cup cheddar cheese, egg...",[],"[basil, cheese, eggs, grapeseed, onions, salt,..."
1465,Zucchini Burgers with Lemon Herb Yogurt Sauce,"[cup bread crumbs, cup Diced cheddar cheese, c...",45,Moderate,https://spoonacular.com/recipeImages/665721-55...,https://recipes.lewagon.com/recipes/524,"[cup bread crumbs, cup diced cheddar cheese, c...",[],"[bread, cheese, parsley, eggs, hamburger, lemo..."
1466,Zucchini Chicken Omelette,"[Eggs, tablespoon Water, grams Zucchini, grate...",45,Moderate,https://spoonacular.com/recipeImages/665734-55...,https://recipes.lewagon.com/recipes/485,"[eggs, tablespoon water, grams zucchini grated...",[],"[eggs, zucchini, salt, pepper, chicken]"
1467,Zucchini Quiche Appetizers,"[cups zucchini, grated, cup Bisquick, cup fine...",45,Hard,https://spoonacular.com/recipeImages/665778-55...,https://recipes.lewagon.com/recipes/720,"[cups zucchini grated, cup bisquick, cup finel...",[],"[zucchini, onion, cheese, parsley, salt, orega..."


In [16]:
data.to_csv('data/Recipes2.csv')

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


class MakeDataFrame:

    def __init__(self):
        self.full_recipe_links = []
        self.foods = [
            # Meats and Fish
            "chicken",
            "beef",
            "pork",
            "lamb",
            "veal",
            "turkey",
            "duck",
            "fish",
            "salmon",
            "tuna",
            "shrimp",
            "mussels",
            "oysters",
            "bacon",
            "ham",

            # Vegetables
            "carrot",
            "potato",
            "tomato",
            "onion",
            "garlic",
            "spinach",
            "broccoli",
            "zucchini",
            "eggplant",
            "bell pepper",
            "mushroom",
            "green beans",
            "asparagus",
            "cabbage",
            "celery",
            "cucumber"

            # Fruits
            "apple",
            "banana",
            "orange",
            "strawberries",
            "grapes",
            "pear",
            "pineapple",
            "kiwi",
            "melon",
            "mango",
            "avocado",
            "cherry",
            "raspberry",
            "blueberry",
            "lime",
            "lemon",
            "coconut"

            # Dairy Products
            "milk",
            "cheese",
            "butter",
            "yogurt",
            "cream",
            "ice cream",

            # Grains
            "rice",
            "pasta",
            "bread",
            "flour",
            "oats",
            "barley",
            "quinoa",

            # Legumes
            "red beans",
            "lentils",
            "chickpeas",
            "fava beans",

            # Herbs and Spices
            "basil",
            "parsley",
            "thyme",
            "rosemary",
            "oregano",
            "cumin",
            "coriander",
            "cinnamon",
            "turmeric",
            "paprika",
            "pepper",
            "salt",

            # Condiments and Sauces
            "ketchup",
            "mustard",
            "mayonnaise",
            "balsamic vinegar",
            "soy sauce",
            "tomato sauce",
            "barbecue sauce",
            "hot sauce",
            "olive oil",
            "vinaigrette",
            "mint"

            # Nuts and Seeds
            "almonds",
            "cashews",
            "walnuts",
            "pistachios",
            "sunflower seeds",
            "sesame seeds",
            "chia seeds",

            # Sweeteners
            "sugar",
            "honey",
            "maple syrup",
            "stevia",
            "syrup",
            "chocolate"

            # Beverages
            "water",
            "coffee",
            "fruit juice",
            "almond milk",
            "coconut milk",

            # Others
            "eggs",
            "egg",
            "almond flour",
            "coconut flour",
            "yeast",
            "sourdough starter"
        ]

        self.stop_words = set(stopwords.words('english'))

    def scrape_recipes_page(self, page_number):
        base_url = "https://recipes.lewagon.com/"
        params = {"page": page_number}

        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            return soup
        else:
            print(f"Error while retrieving page {page_number}.")
            return None

    def extract_preparation_duration(self, description_text):
        duration_pattern = r"(\d+)\s*minutes"
        duration_match = re.search(duration_pattern, description_text)
        if duration_match:
            preparation_duration = duration_match.group(1)
            return preparation_duration
        return None

    def scrape_recipe_info(self, recipe_url):
        response = requests.get(recipe_url)

        if response.status_code == 200:
            recipe_soup = BeautifulSoup(response.content, "html.parser")

            name = recipe_soup.find("h2").text.strip()

            img_element = recipe_soup.find("img",
                                           class_="recipe-img mb-3 mb-md-0")
            img_url = img_element['src'] if img_element else ''

            ingredient_spans = recipe_soup.find_all("p", class_="mb-0")
            ingredients = [span.text.strip() for span in ingredient_spans]

            description_element = recipe_soup.find(
                "p", class_="text-justify recipe-description")
            description_text = description_element.get_text(
            ) if description_element else ""
            duration = self.extract_preparation_duration(description_text)

            difficulty_element = recipe_soup.find("span",
                                                  class_="recipe-difficulty")
            difficulty = difficulty_element.text.strip(
            ) if difficulty_element else ""

            return {
                "name": name,
                "ingredients": ingredients,
                "duration": duration,
                "difficulty": difficulty,
                "img_url": img_url
            }
        else:
            print(f"Error while retrieving recipe {recipe_url}.")
            return None

    def clean_ingredient(self, ingredient):
        cleaned_ingredient = re.sub(r"\d+|/", "", ingredient).strip()
        return cleaned_ingredient

    def preprocess_ingredients(self, ingredient_list):
        processed_ingredients = []
        removed_stopwords = []
        for ingredient in ingredient_list:
            ingredient_lower = ingredient.lower().replace(',',
                                                          '').replace('.', '')
            if ingredient_lower not in self.stop_words:
                processed_ingredients.append(ingredient_lower)
            else:
                removed_stopwords.append(ingredient_lower)
        return processed_ingredients, removed_stopwords

    def real_ingredients(self, ingredients):
        cleaned_ingredients = []
        for ingredient in ingredients:
            ingredient = re.sub(r'(\w+)\)', r'\1', ingredient)
            words = ingredient.split()
            new_words = [
                word for word in words
                if any(food in word for food in self.foods)
            ]
            cleaned_ingredient = ', '.join(new_words)
            
             # Remove any quotation marks present
            cleaned_ingredient = cleaned_ingredient.replace('"', '').replace("'", '')
        
            if cleaned_ingredient:
                cleaned_ingredients.append(cleaned_ingredient)
        return cleaned_ingredients

    def make_dataframe(self, total_pages):
        min_recipes_per_page = 4
        max_recipes_per_page = 13

        # Step 1: Scrape full recipe links
        for page_number in range(1, total_pages + 1):
            list_soup = self.scrape_recipes_page(page_number)

            if list_soup:
                recipe_articles = list_soup.find_all("div",
                                                     class_="recipe my-3")
                num_recipes_on_page = min(
                    max_recipes_per_page,
                    max(min_recipes_per_page, len(recipe_articles)))

                for i in range(num_recipes_on_page):
                    recipe_link = recipe_articles[i]["data-href"]
                    full_recipe_link = recipe_link
                    self.full_recipe_links.append(full_recipe_link)
            else:
                print(
                    f"Unable to retrieve the list of recipes for page {page_number}."
                )

        # Step 2: Scrape individual recipes and store data in recipes_data
        recipes_data = []

        for full_recipe_link in self.full_recipe_links:
            recipe_info = self.scrape_recipe_info(full_recipe_link)
            if recipe_info:
                recipes_data.append(recipe_info)

        data = pd.DataFrame(recipes_data)

        # Step 3: Preprocess ingredients
        data['ingredients'] = data['ingredients'].apply(
            lambda ingredients:
            [clean_ingredient(ingredient) for ingredient in ingredients])

        data['ingredients_processed'], data['removed_stopwords'] = zip(
            *data['ingredients'].apply(preprocess_ingredients))

        # Step 4: Apply real_ingredients function
        data['cleaned_ingredients'] = data['ingredients_processed'].apply(clean_ingredients)

        # Step 5: Save to CSV
        data.to_csv('data/Recipes.csv', index=False)
        
        print("DataFrame updated")  # Added message


if __name__ == "__main__":
    NB_PAGES = 123
    make_dataframe_instance = MakeDataFrame()
    make_dataframe_instance.make_dataframe(NB_PAGES)

DataFrame updated


In [40]:
data = pd.read_csv('data/Recipes.csv')
data

,name,ingredients,duration,difficulty,img_url,ingredients_processed,removed_stopwords,cleaned_ingredients
0,15 Minute Healthy Dark Chocolate Truffles,"['dried dates, pitted', 'cups sliced or sliver...",45.0,Moderate,https://spoonacular.com/recipeImages/631841-55...,"['dried dates pitted', 'cups sliced or slivere...",[],"['honey', 'salt']"
1,17 Bean White Chicken Chili,"['pound Assorted Dry Beans, Cooked According t...",45.0,Hard,https://spoonacular.com/recipeImages/631843-55...,['pound assorted dry beans cooked according to...,[],"['chicken', 'chicken', 'pepper', 'onion', 'gar..."
2,2 Minute Chocolate Yum,"['tablespoons Cocoa Powder', 'Egg, slightly be...",45.0,Very hard,https://spoonacular.com/recipeImages/631849-55...,"['tablespoons cocoa powder', 'egg slightly bea...",[],"['egg', 'flour', 'salt', 'sugar']"
3,3 Steps for the Best Sheet Pan Chicken and Veg...,"['fresh and dried herbs', '- pieces asparagus'...",35.0,Hard,https://spoonacular.com/recipeImages/1045246-5...,"['fresh and dried herbs', '- pieces asparagus'...",[],"['asparagus', 'zucchini', 'chicken', 'cheese',..."
4,4 Ingredient Raw Peanut Butter Chocolate Cups,"['to tablespoons coconut flour, as needed', '...",45.0,Very hard,https://spoonacular.com/recipeImages/792705-55...,"['to tablespoons coconut flour as needed', 'o...",[],"['flour', 'syrup', 'butter']"
...,...,...,...,...,...,...,...,...
1464,Zucchini & Basil Frittata,"['leaves basil, chopped', 'cup cheddar cheese'...",45.0,Moderate,https://spoonacular.com/recipeImages/665691-55...,"['leaves basil chopped', 'cup cheddar cheese',...",[],"['basil', 'cheese', 'eggs', 'grapeseed', 'onio..."
1465,Zucchini Burgers with Lemon Herb Yogurt Sauce,"['cup bread crumbs', 'cup Diced cheddar cheese...",45.0,Moderate,https://spoonacular.com/recipeImages/665721-55...,"['cup bread crumbs', 'cup diced cheddar cheese...",[],"['bread', 'cheese', 'parsley', 'eggs', 'hambur..."
1466,Zucchini Chicken Omelette,"['Eggs', 'tablespoon Water', 'grams Zucchini, ...",45.0,Moderate,https://spoonacular.com/recipeImages/665734-55...,"['eggs', 'tablespoon water', 'grams zucchini g...",[],"['eggs', 'zucchini', 'salt, pepper', 'chicken']"
1467,Zucchini Quiche Appetizers,"['cups zucchini, grated', 'cup Bisquick', 'cup...",45.0,Hard,https://spoonacular.com/recipeImages/665778-55...,"['cups zucchini grated', 'cup bisquick', 'cup ...",[],"['zucchini', 'onion', 'cheese', 'parsley', 'sa..."


In [41]:
import tkinter as tk
from tkinter import messagebox
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from PIL import ImageTk, Image
import requests
import io
import webbrowser
import ast
import pandas as pd

class RecipeRecommendationApp:

    def __init__(self, root):
        self.root = root
        self.root.title("Recipe Recommendation Application")
        
        # Load the DataFrame
        self.clean_data()

        # Set window size (width x height)
        self.root.geometry("1100x700")  # Replace with desired dimensions

        # Set background color of the window
        self.root.configure(bg="#DCDCDC")  # Replace with desired color

        self.create_widgets()
        
    def clean_data(self):
        self.data = pd.read_csv('data/Recipes2.csv')
        self.data = self.data[self.data['duration'].notnull()]
        self.data['duration'] = self.data['duration'].astype(int)
        self.data['difficulty'] = self.data['difficulty'].str.lower()
        self.data.reset_index(inplace=True)
        self.data['cleaned_ingredients'] = self.data['cleaned_ingredients'].apply(ast.literal_eval)
        self.data = self.data.drop(columns=["index", "ingredients", "ingredients_processed", "removed_stopwords"])
    
    def create_widgets(self):
        self.ingredients_label = tk.Label(
            self.root, text="Ingredients (comma-separated):",
            fg="#000080")  # Dark Blue
        self.ingredients_entry = tk.Entry(self.root)
        self.duration_label = tk.Label(
            self.root,
            text="Max Preparation Duration (minutes):",
            fg="#000080")  # Dark Blue
        self.duration_slider = tk.Scale(self.root,
                                        from_=5,
                                        to=55,
                                        orient="horizontal",
                                        fg="#000080")  # Dark Blue
        self.difficulty_label = tk.Label(self.root,
                                         text="Preparation Difficulty:",
                                         fg="#000080")  # Dark Blue
        self.difficulty_var = tk.StringVar()
        self.difficulty_dropdown = tk.OptionMenu(self.root,
                                                 self.difficulty_var,
                                                 "very easy", "easy",
                                                 "moderate", "hard",
                                                 "very hard")
        self.recommend_button = tk.Button(self.root,
                                          text="Recommend Recipes",
                                          command=self.recommend_recipes,
                                          fg="#000080")  # Dark Blue

        # Pack widgets
        self.ingredients_label.pack()
        self.ingredients_entry.pack()
        self.duration_label.pack()
        self.duration_slider.pack()
        self.difficulty_label.pack()
        self.difficulty_dropdown.pack()
        self.recommend_button.pack()

        # Create a frame to display results (initially empty)
        self.results_frame = tk.Frame(self.root)
        self.results_frame.pack()

    def recommend_recipes(self):
        # Extract user input and preferences
        ingredients = self.ingredients_entry.get().split(',')
        max_duration = self.duration_slider.get()
        desired_difficulty = self.difficulty_var.get().lower()

        # Instantiate and fit the TfidfVectorizer
        vectorizer = TfidfVectorizer()
        ingredients_matrix = vectorizer.fit_transform([
            ' '.join(ingredients)
            for ingredients in self.data['cleaned_ingredients']
        ])

        # User-provided ingredients
        user_ingredients_vector = vectorizer.transform([' '.join(ingredients)])

        # Calculate cosine similarity
        similarities = cosine_similarity(user_ingredients_vector,
                                         ingredients_matrix).flatten()

        filtered_recipes = []
        for idx, recipe in enumerate(self.data['name']):
            if self.data['duration'][idx] <= max_duration and self.data['difficulty'][
                    idx] == desired_difficulty:
                filtered_recipes.append((recipe, similarities[idx]))

        filtered_recipes.sort(key=lambda x: x[1], reverse=True)
        
        # Display results in the results frame
        self.results_frame.destroy()
        self.results_frame = tk.Frame(self.root)
        self.results_frame.pack()

        # Display results in the results frame
        for idx, (recipe, similarity) in enumerate(filtered_recipes[:3]):
            if similarity > 0:
                URL = self.data[self.data['name'] == recipe]['URL'].values[0]
                url = self.data[self.data['name'] == recipe]['img_url'].values[0]
                img_data = requests.get(url).content
                img = Image.open(io.BytesIO(img_data))
                img = img.resize((150, 150), Image.ANTIALIAS)
                img = ImageTk.PhotoImage(img)

                recipe_text = f"{idx+1}. {recipe} (Similarity : {similarity:.2f})"
                recipe_label = tk.Label(
                    self.results_frame,
                    text=recipe_text,
                    fg="#0000FF",  # Blue color for hyperlink appearance
                    cursor="hand2"  # Change cursor to hand on hover
                )
                recipe_label.pack()

                # Bind the callback to the label (outside the loop)
                recipe_label.bind("<Button-1>", lambda event, url=URL: self.open_url(event, url))

                # Display recipe image
                img_label = tk.Label(self.results_frame, image=img)
                img_label.image = img
                img_label.pack()
            else:
                # Display message when no matching recipes
                no_recipe_label = tk.Label(
                    self.results_frame, text="No recipes match your criteria.")
                no_recipe_label.pack()

    def open_url(self, event, url):
        webbrowser.open(url)  # Open URL in default web browser

if __name__ == "__main__":
    root = tk.Tk()
    app = RecipeRecommendationApp(root)
    root.mainloop()


/tmp/ipykernel_3953/1944272575.py:119: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((150, 150), Image.ANTIALIAS)
